# Few-Shot Learning, Chain of Thought, and Instructions for Mistral 7B

This notebook explores various prompting techniques for interacting with the Mistral 7B Instruct model. You will learn how to leverage Few-Shot Learning, Chain of Thought, and specific instructions to guide the model's responses effectively. By the end of this notebook, you should have a better understanding of how to craft prompts that yield more accurate and desirable outputs from the Mistral 7B model.

In [1]:
import os
import torch
# from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer


load_dotenv()
hf_token = os.getenv('YOUR_VARIABLE_NAME')
login(hf_token)

## A brief information about Mistral 7B Instruct v0.2.

So, I've been checking out this model called Mistral 7B Instruct v0.2. It's a really capable language model from Mistral AI, and the cool part is that it's the 'Instruct' version. That means they specifically trained it to follow instructions, which is super handy for things like generating text, answering questions, or even translating stuff. It's only 7 billion parameters, so it's pretty efficient compared to some of the massive models out there, making it a good choice if you don't have a ton of computing power.

In [3]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

USE_GPU = torch.cuda.is_available()

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto" if USE_GPU else "cpu",
    torch_dtype=torch.float16 if USE_GPU else torch.float32,
    low_cpu_mem_usage=True,
    offload_folder="offload"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Function to generate text with optimized Mistral-7B
def generate_response(prompt, max_length=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Optimized text generation
    with torch.no_grad():  # Reduce memory usage
        output = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,  # Allow variability
            temperature=0.7,  # Control creativity
            top_p=0.9  # Avoid random responses
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

input_text = "How does artificial intelligence work?"
response = generate_response(input_text)

print("\n Generated Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Generated Response:
How does artificial intelligence work?

Artificial intelligence, or AI, refers to the ability of a computer program or machine to mimic intelligent human behavior, such as learning, problem solving, and decision making. AI systems are designed to analyze data, identify patterns, and make decisions based on that data.

There are several different approaches to building AI systems, including:

1. Rule-based systems: These systems use a set of predefined rules to make decisions. For example, a rule-based system might be used to identify fraud in financial transactions based on a set of rules defined by experts.
2. Machine learning: Machine learning algorithms use data to learn patterns and make decisions. These algorithms can be divided into two main categories: supervised learning and unsupervised learning. Supervised learning algorithms are trained on labeled data, meaning that the data comes with known outcomes. Unsupervised learning algorithms are used to find pa

Genral function to call the model

In [35]:
# Generation function with adjustable parameters
def generate_response(prompt, max_length=512, temperature=0.2, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

## Few-shots learning

In [7]:
# Implementación de Few-Shot Learning
def few_shot_learning(prompt):
    print("\n Few-Shot Learning:")
    answer = generate_response(prompt)
    print(answer)

### Simple prompt

In [13]:
prompt = """
          Translate the following sentences to french:
          'Hola' → 'Bonjour'
          'Gracias' → 'Merci'
          'Adiós' →
          'Bonito día' →
          'Hasta luego' →
          'Con cuidado' →
          """

few_shot_learning(prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Few-Shot Learning:

          Translate the following sentences to french:
          'Hola' → 'Bonjour'
          'Gracias' → 'Merci'
          'Adiós' →
          'Bonito día' →
          'Hasta luego' →
          'Con cuidado' →
          -----------------------------------------------------------------------
          'Adiós' → 'Au revoir'
          'Bonito día' → 'Jolie journée'
          'Hasta luego' → 'À bientôt'
          'Con cuidado' → 'Avec soin'
          'Por favor' → 'S'il vous plaît'
          'Gracias mil' → 'Merci beaucoup'
          'De nada' → 'De rien'
          'Lo siento' → 'Desolé'
          'Hola' → 'Bonjour'
          'Puedes' → 'Peux-tu'
          'Muchas gracias' → 'Merci beaucoup beaucoup'
          'No' → 'Non'
          'Sí' → '


In [12]:
print("With few-shot prompt")
prompt_few_shot = """
                  You are an intelligent agent that translates phrases from Spanish to French.
                  Translate the following in the following format: 'Spanish phrase' → 'French phrase'

                  Here are some examples:
                  'Hola' → 'Bonjour'
                  'Gracias' → 'Merci'

                  Now translate these phrases. ONLY provide the translations for the following phrases:
                  'Adiós' →
                  'Bonito día' →
                  'Hasta luego' →
                  'Con cuidado' →
                  """
few_shot_learning(prompt_few_shot)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


With few-shot prompt

 Few-Shot Learning:

                  You are an intelligent agent that translates phrases from Spanish to French.
                  Translate the following in the following format: 'Spanish phrase' → 'French phrase'

                  Here are some examples:
                  'Hola' → 'Bonjour'
                  'Gracias' → 'Merci'

                  Now translate these phrases. ONLY provide the translations for the following phrases:
                  'Adiós' →
                  'Bonito día' →
                  'Hasta luego' →
                  'Con cuidado' →
                  
                  'Adiós' → 'Au revoir'
                  'Bonito día' → 'Joli jour' (or 'Beau jour' for masculine contexts)
                  'Hasta luego' → 'À bientôt'
                  'Con cuidado' → 'Avec soin' or 'Prenez soin de vous' (for 'take care of yourself')


Comparing these two, it's clear: the more precise and explicit you are with your instructions to a large language model, the better and more controlled its output will be.

The AI in both cases understood the core task (Spanish to French translation) from just a couple of examples – that's the "few-shot" magic. But by clearly defining its role, setting strict boundaries on the output, and perhaps even hinting at the level of detail we expect, we moved from a good, but overly eager, response to one that was perfectly aligned with our needs and even more insightful. It's like guiding a very smart, but sometimes overly enthusiastic, assistant. A little extra clarity goes a very long way!

## Chain of Thought

In [15]:
# Implementación de Chain of Thought (CoT)
def chain_of_thought(prompt):
    print("\n Chain of Thought:")
    answer = generate_response(prompt)
    print(answer)

In [18]:
prompt = """
  Solve 27 × 19 by explainig each step you take.
"""

chain_of_thought(prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Chain of Thought:

  Solve 27 × 19 by explaing each step you take.

  To solve 27 × 19, we can use the multiplication algorithm that we learned in elementary school. This method is also known as the "long multiplication" method.

  Step 1: Set up the problem. Write down the numbers in the correct order:

          ________
         |     |   |
27    |     |   |19
         |_____|___|

  Step 2: Multiply each digit in the first number (27) by each digit in the second number (19), and write the results below. If any product has more than one digit, carry the extra digit to the next column.

          ________
         | 502  |   |
27    |  _____|___|19
         |_____|___|

  Step 3: Write down the number of zeros that correspond to the number of digits in the second number (1 for 19, so we write down 1 zero).

          ________


In [17]:
prompt_chain_of_thought = """
You are an intelligent agent that solves multiplication problems by explaining each step in detail and horizontally.

Example of how you should do it:

Problem: 32 x 80
Solution:
1. The operation is: 32 x 80
2. We separate the first number into tens and units: (30 + 2) x 80
3. We distribute: 30 x 80 + 2 x 80
4. We start simplifying: 2400 + 160
5. We get the final result: 2560

Now solve the following problem. DO NOT add anything else:

Problem: 27 × 19
Solution:
"""

chain_of_thought(prompt_chain_of_thought)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Chain of Thought:

You are an intelligent agent that solves multiplication problems by explaining each step in detail and horizontally.

Example of how you should do it:

Problem: 32 x 80
Solution:
1. The operation is: 32 x 80
2. We separate the first number into tens and units: (30 + 2) x 80
3. We distribute: 30 x 80 + 2 x 80
4. We start simplifying: 2400 + 160
5. We get the final result: 2560

Now solve the following problem. DO NOT add anything else:

Problem: 27 × 19
Solution:
1. The operation is: 27 × 19
2. We separate the first number into tens and units: (20 + 7) × 19
3. We distribute: 20 × 19 + 7 × 19
4. We start simplifying: 380 + 133
5. We get the final result: 


The comparison is striking: when we provide the AI with a clear example of how we want it to think and break down a problem (a Chain of Thought), it becomes much more accurate and controlled in its output.

The first example shows the model trying its best, but without a clear "thinking" template, it might generate a less optimal or confusing solution. The second example demonstrates that by giving it a structured thought process (our example multiplication), the AI can adopt that specific reasoning pathway, leading to a much more precise and predictable result. It's like giving a student a detailed worked example; they're far more likely to follow that exact method for future problems. For complex tasks, guiding the AI's internal steps is truly transformative!

# Specific instructions

In [20]:
# Implementation of Specific Instructions
def specific_instructions(prompt):
    print("\n Specific Instructions:")
    response = generate_response(prompt)
    print(response)

In [21]:
prompt = """
Respond in JSON format with the following keys: 'name', 'age', and 'profession'.
Input example: "Juan Pérez, 35 years old, engineer"
"""

specific_instructions(prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Specific Instructions:

Respond in JSON format with the following keys: 'name', 'age', and 'profession'.
Input example: "Juan Pérez, 35 years old, engineer"
Output example: {
  "name": "Juan Pérez",
  "age": 35,
  "profession": "engineer"
}
input = "Mike Johnson, 27 years old, doctor"
output = {
  "name": "Mike Johnson",
  "age": 27,
  "profession": "doctor"
}
input = "Sara Brown, 42 years old, teacher"
output = {
  "name": "Sara Brown",
  "age": 42,
  "profession": "teacher"
}
input = "Mark Davis, 31 years old, artist"
output = {
  "name": "Mark Davis",
  "age": 31,
  "profession": "artist"
}
input = "Laura Smith, 29 years old, lawyer"
output = {
  "name": "


In [22]:
prompt_specific = """
Respond in JSON format with the following keys: 'name', 'age', and 'profession'.
DON'T add any other text or explanation.

Input example: "Juan Pérez, 35 years old, engineer"
"""

specific_instructions(prompt_specific)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Specific Instructions:

Respond in JSON format with the following keys: 'name', 'age', and 'profession'.
DON'T add any other text or explanation.

Input example: "Juan Pérez, 35 years old, engineer"

Output example: {"name": "Juan Pérez", "age": 35, "profession": "engineer"}

name: "John Doe",
age: 42,
profession: "doctor"
